In [9]:
import sys

sys.path.append("../..")

from copy import deepcopy
import omegaconf
import torch
import pandas as pd
from next_sentence.utils import FrozenModelSentenceGivenPrompt, ReplayBuffer
from next_sentence.lightning_module import NextSentenceGFNTask
from train_next_sentence import get_model

torch.set_grad_enabled(False)

In [17]:
config = omegaconf.DictConfig(
    {
        "task": {
            "model": {
                "name": "distilgpt2",
                "lora_config": {
                    "_target_": "peft.LoraConfig",
                    "target_modules": ["c_attn", "c_proj", "c_fc"],
                    "r": 64,
                    "lora_alpha": 16,
                    "lora_dropout": 0.1,
                    "bias": "none",
                    "fan_in_fan_out": True,
                },
            }
        }
    }
)

model, tokenizer = get_model(config)
model_orig_ = deepcopy(model).to("mps")
model = NextSentenceGFNTask.load_from_checkpoint(
    "~/Downloads/best.ckpt", model=model, tokenizer=tokenizer, map_location="mps"
)
model_orig = deepcopy(model)
model_orig.model = model_orig_
del model_orig_

model.eval()
model_orig.eval()
model.reward.temperature = 1.0
model_orig.reward.temperature = 1.0

In [19]:
prompts = [
    "My dog is black. ",
    "I've been working on this far too long. ",
    "There's something I need to get off my chest. ",
]
prompts_tensor = [
    tokenizer(prompt, return_tensors="pt").input_ids[0].to(model.device)
    for prompt in prompts
]

In [79]:
samples = model.sample_probes(prompts_tensor, n_samples=1)

In [22]:
samples_orig = model_orig.sample_probes(prompts_tensor, n_samples=1)

In [80]:
samples

,Prompt,Sampled sentence,logP(s),Sampled sentence (beam),logP(s) (beam)
2,There's something I need to get off my chest.,????,-8.783124,ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ�,-24.739201
0,My dog is black.,________,-9.519690,__________________\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,-20.008408
1,I've been working on this far too long.,\nhttp://www,-11.228891,_________________\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,-21.900307


In [ ]:
samples_orig